In [169]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# Read csv and save to parquet. Needs to be done only once
# transactions = pd.read_csv('data/synt_transactions_10M.csv', sep=';')
# transactions.to_parquet('data/transactions_categorical.parquet')

In [30]:
transactions = pd.read_parquet('data/transactions_categorical.parquet')
display(transactions)
display(transactions)

,category,tstamp,arvopvm,kirjauspvm,maksupvm,tilinro,rahamaara,saldo,vientiselitekd,taplajikd,bic_saaja,viite,iban_saaja,counterparty_account_id
0,Tulot,2019-01-01 00:00:00.000000000,2019-01-01,2019-01-01,2019-01-01,1.0,22.09,59.55,0.0,506.0,,1.0,1.0,177.0
1,Ruoka_paivittaistavarakauppa,2019-01-01 13:54:23.551401869,2019-01-01,2019-01-02,2019-01-01,1.0,-9.47,50.08,0.0,162.0,,1.0,1.0,116.0
2,Ruoka_paivittaistavarakauppa,2019-01-02 03:48:47.102803738,2019-01-02,2019-01-02,2019-01-02,1.0,-16.77,33.31,0.0,162.0,,1.0,1.0,137.0
3,Ruoka_paivittaistavarakauppa,2019-01-02 17:43:10.654205607,2019-01-02,2019-01-02,2019-01-02,1.0,-0.89,32.42,0.0,162.0,,1.0,1.0,33.0
4,Ruoka_paivittaistavarakauppa,2019-01-03 07:37:34.205607476,2019-01-03,2019-01-03,2019-01-03,1.0,-5.93,26.49,0.0,162.0,,1.0,1.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936641,Asuminen,2019-07-05 10:01:09.902912619,2019-07-05,2019-07-05,2019-07-05,9155.0,-26.21,935.10,706.0,129.0,OKOYFIHH,0.0,0.0,234.0
9936642,Ravintolat_kahvilat,2019-07-05 19:34:22.135922328,2019-07-05,2019-07-05,2019-07-05,9155.0,-11.84,923.26,0.0,162.0,,1.0,1.0,43.0
9936643,Tulot,2019-07-08 05:21:33.203883492,2019-07-08,2019-07-08,2019-07-08,9155.0,124.01,1047.27,710.0,588.0,,0.0,1.0,252.0
9936644,Shoppailu,2019-07-08 14:54:45.436893201,2019-07-08,2019-07-08,2019-07-08,9155.0,-17.80,1029.47,0.0,162.0,,1.0,1.0,128.0


In [124]:
incomes = transactions.loc[transactions['category'] == 'Tulot']
account_grouped = transactions.groupby('tilinro')
min_dates = account_grouped['maksupvm'].min()
max_dates = account_grouped['maksupvm'].max()
date_deltas = max_dates - min_dates
monthly_incomes = transactions.loc[transactions['category'] == 'Tulot'].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 12
living_costs = -transactions.loc[transactions['category'] == 'Asuminen'].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 127
living_costs = -transactions.loc[transactions['category'] == 'Asuminen'].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 12

display(living_costs)

tilinro
1.0         37.969930
2.0        290.620833
3.0       1270.659583
4.0        263.995417
5.0        128.022500
             ...     
9151.0     358.481250
9152.0     117.857083
9153.0     212.798171
9154.0     268.432083
9155.0     158.569526
Length: 9155, dtype: float64

In [9]:
example = transactions.loc[transactions['tilinro'] == 1]


In [45]:
example.loc[:, 'maksupvm'] = pd.to_datetime(example['maksupvm'])
transactions = transactions.dropna()

transactions.loc[:, 'maksupvm'] = pd.to_datetime(transactions['maksupvm'])

# print((example['maksupvm'].dt.month == 2) and (example['maksupvm'].dt.year == 2020))
month = example.loc[(example['maksupvm'].dt.month == 2) & (example['maksupvm'].dt.year == 2020)]



In [123]:
display(pd.Series(transactions['category'].unique()))

0                            Tulot
1     Ruoka_paivittaistavarakauppa
2                      Liikkuminen
3                 Kulttuuri_viihde
4                             Muut
5                         Asuminen
6               Luokittelemattomat
7                         Matkailu
8                 Luottojen_maksut
9              Saastot_sijoitukset
10             Ravintolat_kahvilat
11                     Vakuutukset
12                         Terveys
13                    Harrastukset
14            Hyvinvointijakauneus
15                        Lemmikit
16                     Hyvinvointi
17                          Lapset
18                       Shoppailu
dtype: object

In [125]:
%matplotlib widget
plt.close('all')
# plt.plot(month['maksupvm'], month['saldo'])
# plt.grid()
# plt.show()
plt.hist(yearly_incomes, bins='auto')
print(incomes.median())
display(transactions['maksupvm'].max())
display(transactions['maksupvm'].min())
# transactions.isna().sum()
(transactions['maksupvm'].max()-transactions['maksupvm'].min()).days
# plt.xscale('log')
monthly_incomes.median()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-125-c1e58b92067b>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  print(incomes.median())


tilinro                    4582.00
rahamaara                   125.29
saldo                      1511.95
vientiselitekd              701.00
taplajikd                   520.00
viite                         1.00
iban_saaja                    1.00
counterparty_account_id     205.00
dtype: float64


Timestamp('2021-01-04 00:00:00')

Timestamp('2018-12-27 00:00:00')

1664.138333333334

In [110]:
def color_bar(bar_value_to_label, patches):
    min_distance = float("inf")  # initialize min_distance with infinity
    index_of_bar_to_label = 0
    for i, rectangle in enumerate(patches):  # iterate over every bar
        tmp = abs(  # tmp = distance from middle of the bar to bar_value_to_label
            (rectangle.get_x() +
                (rectangle.get_width() * (1 / 2))) - bar_value_to_label)
        if tmp < min_distance:  # we are searching for the bar with x cordinate
                                # closest to bar_value_to_label
            min_distance = tmp
            index_of_bar_to_label = i
    patches[index_of_bar_to_label].set_color('r')

In [162]:
%matplotlib widget
plt.close('all')
customer_id = 11
window_size = 100
customer = monthly_incomes[customer_id]
sorted_incomes = yearly_incomes.sort_values()
customer_ranking = sorted_incomes.index.get_loc(customer_id)
similar_customers = sorted_incomes.iloc[customer_ranking-window_size:customer_ranking+window_size].index
similar_living_costs = living_costs.loc[similar_customers]
similar_living_costs = similar_living_costs.loc[(similar_living_costs < similar_living_costs.quantile(0.99)) & (similar_living_costs > similar_living_costs.quantile(0.01))]
n, bins, patches = plt.hist(similar_living_costs, bins='auto')
color_bar(living_costs.loc[customer_id], patches)
title = 'similar_livin_'
plt.title(monthly_incomes.loc[customer_id])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '1857.0384716506635')

In [177]:

customer_transactions = transactions.loc[transactions['tilinro'] == customer_id][['maksupvm', 'saldo']]
display(customer_transactions)
customer_transactions_weekly = customer_transactions.resample('D', on='maksupvm').saldo.mean().interpolate('time')
%matplotlib widget
plt.plot(customer_transactions['maksupvm'], customer_transactions['saldo'])
# customer_transactions_weekly.maksupvm.duplicated()
plt.plot(customer_transactions_weekly)
display(customer_transactions_weekly)
customer_transactions_weekly.isna().sum()




,maksupvm,saldo
10533,2019-01-01,4950.58
10534,2019-01-08,4620.84
10535,2019-01-14,6104.41
10536,2019-01-18,6092.16
10537,2019-01-24,6005.01
...,...,...
11069,2020-12-28,14734.17
11070,2020-12-28,14727.01
11071,2020-12-28,14643.40
11072,2020-12-30,14547.94


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

maksupvm
2019-01-01     4950.580000
2019-01-02     4903.474286
2019-01-03     4856.368571
2019-01-04     4809.262857
2019-01-05     4762.157143
                  ...     
2020-12-26    14640.283611
2020-12-27    14655.050556
2020-12-28    14669.817500
2020-12-29    14775.193750
2020-12-30    14880.570000
Freq: D, Name: saldo, Length: 730, dtype: float64

0

In [186]:
#preprocess

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(customer_transactions_weekly.values.astype('float32').reshape(-1, 1))

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))



489 241


In [191]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)


# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [192]:

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
487/487 - 0s - loss: 0.0228
Epoch 2/100
487/487 - 0s - loss: 0.0174
Epoch 3/100
487/487 - 0s - loss: 0.0137
Epoch 4/100
487/487 - 0s - loss: 0.0096
Epoch 5/100
487/487 - 0s - loss: 0.0055
Epoch 6/100
487/487 - 0s - loss: 0.0022
Epoch 7/100
487/487 - 0s - loss: 5.8880e-04
Epoch 8/100
487/487 - 0s - loss: 1.7902e-04
Epoch 9/100
487/487 - 0s - loss: 1.3821e-04
Epoch 10/100
487/487 - 0s - loss: 1.3677e-04
Epoch 11/100
487/487 - 0s - loss: 1.3431e-04
Epoch 12/100
487/487 - 0s - loss: 1.3807e-04
Epoch 13/100
487/487 - 0s - loss: 1.4195e-04
Epoch 14/100
487/487 - 0s - loss: 1.3967e-04
Epoch 15/100
487/487 - 0s - loss: 1.4421e-04
Epoch 16/100
487/487 - 0s - loss: 1.4366e-04
Epoch 17/100
487/487 - 0s - loss: 1.4495e-04
Epoch 18/100
487/487 - 0s - loss: 1.4495e-04
Epoch 19/100
487/487 - 0s - loss: 1.4550e-04
Epoch 20/100
487/487 - 0s - loss: 1.4323e-04
Epoch 21/100
487/487 - 0s - loss: 1.3828e-04
Epoch 22/100
487/487 - 0s - loss: 1.5857e-04
Epoch 23/100
487/487 - 0s - loss: 1.4672e-0